In [1]:
import sys
# sys.path.append("../")
import os
os.chdir("../")

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] ="-1"

In [4]:
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer
from splade.models.transformer_rep import Splade, DebugSplade

In [5]:
from splade.utils.utils import restore_model

In [7]:

model_type_or_dir = "experiments/Only_psuedo/checkpoint/model"
# /home/collab/u1368791/largefiles/TaoFiles/splade/experiments/cocondenser_ensemble_distil/checkpoint/model
psuedo_model = DebugSplade(model_type_or_dir, agg="max")
psuedo_model.eval()
config={}
config["checkpoint_dir"]= "experiments/Only_psuedo/checkpoint"
checkpoint = torch.load(os.path.join(config["checkpoint_dir"], "model_ckpt/model_ckpt_6000.tar"),
                                            map_location="cpu")
restore_model(psuedo_model, checkpoint["model_state_dict"])

tokenizer = AutoTokenizer.from_pretrained(model_type_or_dir)
reverse_voc = {v: k for k, v in tokenizer.vocab.items()}

restoring model: DebugSplade


In [8]:
model_type_or_dir = "experiments/cocondenser_ensemble_distil/checkpoint/model"
# /home/collab/u1368791/largefiles/TaoFiles/splade/experiments/cocondenser_ensemble_distil/checkpoint/model
model = DebugSplade(model_type_or_dir, agg="max")
model.eval()
config={}
config["checkpoint_dir"]= "experiments/cocondenser_ensemble_distil/checkpoint"
checkpoint = torch.load(os.path.join(config["checkpoint_dir"], "model/model.tar"),
                                            map_location="cpu")
restore_model(model, checkpoint["model_state_dict"])

restoring model: DebugSplade


In [43]:
def outputEmb(doc,model=None,returnRep=False):
    with torch.no_grad():
        doc_rep = model(d_kwargs=tokenizer(doc, return_tensors="pt"))["d_rep"].squeeze()  # (sparse) doc rep in voc space, shape (30522,)
    # get the number of non-zero dimensions in the rep:
    col = torch.nonzero(doc_rep).squeeze().cpu().tolist()
    # print("number of actual dimensions: ", len(col))

    # now let's inspect the bow representation:
    weights = doc_rep[col].cpu().tolist()
    # print(col,weights)
    if not isinstance(col,list):
        col=[col]
    # if weights is not list:
        weights=[weights]
    # if source is not list:
        # source=[source]
    # print(col,weights)
    d = {k: v for k, v in zip(col, weights)}
    sorted_d = {k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)}
    bow_rep = []
    for k, v in sorted_d.items():
        bow_rep.append((reverse_voc[k], round(v, 2)))
    # print("SPLADE BOW rep:------", bow_rep)
    if returnRep:
        return bow_rep,doc_rep
    return bow_rep

def outputEmb_source(doc,model=None,returnRep=False):
    with torch.no_grad():
        outputR=model(d_kwargs=tokenizer(doc, return_tensors="pt"))
        doc_rep = outputR["d_rep"].squeeze()  # (sparse) doc rep in voc space, shape (30522,)
        argmax = outputR["argmax"].squeeze() 
    # get the number of non-zero dimensions in the rep:
    col = torch.nonzero(doc_rep).squeeze().cpu().tolist()
    # print("number of actual dimensions: ", len(col))
    tokken=tokenizer(doc, return_tensors="pt")['input_ids']
    # now let's inspect the bow representation:
    weights = doc_rep[col].cpu().tolist()
    argmax = argmax[col].cpu().tolist()
    source=tokken[0][argmax].cpu().tolist()
    if not isinstance(col,list):
        col=[col]
    # if weights is not list:
        weights=[weights]
    # if source is not list:
        source=[source]
    d = {k: (v,j) for k, v,j in zip(col, weights,source)}
    sorted_d = {k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)}
    bow_rep = []
    for k, v in sorted_d.items():
        bow_rep.append((reverse_voc[k],reverse_voc[v[1]], round(v[0], 2)))
    # print("SPLADE BOW rep:------", bow_rep)
    if returnRep:
        return bow_rep,doc_rep
    return bow_rep


In [10]:
# q_id= '443396'
# # "the query is :who is robert gray"
# queryText=q_collection.data_dict[q_id]
# queryText="who"
queryText="define extreme"
print(f"the query is :{queryText}")
top_k=10

queryExplicit,query=outputEmb(queryText,model=model,returnRep=True)
print(queryExplicit)

the query is :define extreme
[('extreme', 37.63), ('definition', 22.9), ('word', 21.48), ('wave', 21.34), ('malcolm', 21.32), ('severe', 21.18), ('express', 21.17), ('stark', 21.09), ('extremes', 21.07), ('austin', 20.97), ('fear', 20.91), ('anger', 20.87), ('rogers', 20.83), ('physics', 20.8), ('savage', 20.78), ('meaning', 20.75), ('turner', 20.52), ('psychology', 20.46), ('mountain', 20.45), ('logic', 20.45), ('literary', 20.44), ('##bari', 20.39), ('comfort', 20.38), ('define', 20.38), ('mental', 20.36), ('parker', 20.36), ('norm', 20.36), ('max', 20.34), ('personality', 20.33), ('baxter', 20.31), ('nature', 20.28), ('epic', 20.26), ('graham', 20.25), ('walter', 20.23), ('strength', 20.21), ('jordan', 20.21), ('wes', 20.18), ('stage', 20.18), ('experiment', 20.17), ('marty', 20.14), ('spencer', 20.14), ('morris', 20.14), ('steve', 20.13), ('alex', 20.13), ('reed', 20.11), ('reid', 20.09), ('respect', 20.09), ('science', 20.09), ('political', 20.08), ('palmer', 20.08), ('liberty', 2

/home/taoyang/miniconda3/envs/splade/lib/python3.8/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [ ]:
from transformers import AutoModel

# model = AutoModel.from_pretrained("bert-base-uncased")
# # Print the weights before and after the call to _init_weights to confirm they have be re-initialized
# # print(model.encoder.layer[-1].attention.output.dense.weight)
# model.encoder.layer[-1].apply(model._init_weights)

In [ ]:
model

DebugSplade(
  (transformer_rep): TransformerRep(
    (transformer): BertForMaskedLM(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
                )
                (output): BertSelfOutput(
        

In [ ]:
for i in model.parameters():
    print(i.shape)
    print(dir(i))

In [ ]:
query

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [ ]:
# import pickle
# TopK=10
# with open('output/log-doc-tf-topk-10.pkl', 'rb') as f:
#     doc_tf = pickle.load(f)

# with open('output/corpus-tf-log-tensor-dictNP.pkl', 'rb') as f:
#     cortf_Rep=pickle.load(f)
# # cortf_Rep=cortf_Rep.numpy()

In [ ]:
import numpy as np


In [ ]:
# q_id='813472'
# topic_Rep=doc_tf[int(q_id)]
# topic_RepNp=np.zeros_like(cortf_Rep).astype(np.float32)
# ind=list(topic_Rep.keys())
# values=list(topic_Rep.values())
# #queryRep1=self.quer_Rep[ind]
# topic_RepNp*=0.0
# topic_RepNp[ind]=values
# numDocs=8*10**6
# psuedo_topk=10
# FinalTopic_Rep=-1/(cortf_Rep+1e-2)
# # FinalTopic_Rep=-(cortf_Rep-topic_RepNp)/(numDocs-psuedo_topk)+topic_RepNp/psuedo_topk
# # FinalTopic_Rep=topic_RepNp/(cortf_Rep+1e-5)
# MiddleTopic_Rep=topic_RepNp**2/(cortf_Rep+1e-2)*numDocs/psuedo_topk
# FinalTopic_Rep[ind]=MiddleTopic_Rep[ind]

In [ ]:
# tripExp=[]
# for t in topic_Rep:
#     # print(reverse_voc[t],topic_Rep[t],FinalTopic_Rep[t])
#     tripExp.append([t,reverse_voc[t],topic_Rep[t],cortf_Rep[t],FinalTopic_Rep[t]])
# tripExp=sorted(tripExp,key=lambda x:-x[-1])
# print(tripExp)

In [ ]:
# exp=[]
# for ind,i in enumerate(FinalTopic_Rep):
#     exp.append((i,reverse_voc[ind]))
# exp=sorted(exp,key=lambda x:x[0])
# print(exp[:100])
# print(exp[-100:])

In [ ]:
import yaml
# opening a file
yamlFile="/home/taoyang/research/research_everyday/projects/DR/splade/splade/experiments/Only_psuedo/checkpoint/config.yaml"
with open(yamlFile, 'r') as stream:
    # try:
    # Converts yaml document to python object
    exp_dictOrig=yaml.safe_load(stream)

In [ ]:
exp_dictOrig["data"]['TRAIN']

{'DATASET_PATH': 'data/msmarco/hard_negatives_scores/cross-encoder-ms-marco-MiniLM-L-6-v2-scores.pkl.gz',
 'D_COLLECTION_PATH': 'data/msmarco/full_collection',
 'Q_COLLECTION_PATH': 'data/msmarco/train_queries/queries',
 'QREL_PATH': 'data/msmarco/train_queries/qrels.json',
 'queryRepFile': 'output/log-doc-tf-topk-10.pkl',
 'corpusRepFile': 'output/corpus-tf-log-tensor-dictNP.pkl',
 'psuedo-topk': 10}

Preloading dataset


0it [00:00, ?it/s]

Preloading dataset


0it [00:00, ?it/s]

QUERY SIZE =  1000


In [11]:
from splade.datasets.datasets import PairsDatasetPreLoad, DistilPairsDatasetPreLoad, MsMarcoHardNegatives, \
    CollectionDatasetPreLoad,MsMarcoHardNegativesWithPsuedo

In [ ]:
inputDict={
    "dataset_path": 'data/msmarco/hard_negatives_scores/cross-encoder-ms-marco-MiniLM-L-6-v2-scores.pkl.gz',
    "document_dir": 'data/msmarco/full_collection',
    "query_dir":'data/msmarco/train_queries/queries',
    "qrels_path":'data/msmarco/train_queries/qrels.json',
    "queryRepFile":'output/log-doc-tf-topk-10.pkl',
    "corpusRepFile":'output/corpus-tf-log-tensor-dictNP.pkl',
    "psuedo_topk":10}
dataset=MsMarcoHardNegativesWithPsuedo(**inputDict)

In [12]:
inputDict={
    "dataset_path": 'data/toy_data1k/hard_negatives_scores/cross-encoder-ms-marco-MiniLM-L-6-v2-scores.pkl.gz',
    "document_dir": 'data/toy_data1k/full_collection',
    "query_dir":'data/toy_data1k/train_queries/queries',
    "qrels_path":'data/msmarco/train_queries/qrels.json',
    "queryRepFile":'data/toy_data1k/psuedo/log-doc-tf-topk-10.pkl',
    "corpusRepFile":'output/corpus-tf-log-tensor-dictNP.pkl',
    "psuedo_topk":10,
    # "toy":True
    }
dataset=MsMarcoHardNegativesWithPsuedo(**inputDict)

Preloading dataset


0it [00:00, ?it/s]

Preloading dataset


0it [00:00, ?it/s]

QUERY SIZE =  1000


In [ ]:
# qselect=dataset.query_list[:1000]
# Toyquerys=[[str(qid),dataset.query_dataset.data_dict[str(qid)]] for qid in qselect]
# List2Tsv(Toyquerys,outputDir="data/toy_data1k/dev_queries/raw.tsv")

# ToyDocs=[]
# for qid,query in Toyquerys:
#     ToyDocs+=list(dataset.scores_dict[int(qid)].keys())
# ToyDocs=list(set(ToyDocs))
# ToyDocs=[[did,dataset.document_dataset.data_dict[str(did)]]for did in ToyDocs]

# List2Tsv(ToyDocs,outputDir="data/toy_data1k/full_collection/raw.tsv")

# subscoreDict={qid:dataset.scores_dict[int(qid)] for qid,query in Toyquerys}
# import pickle
# with open("data/toy_data1k/cross-encoder-ms-marco-MiniLM-L-6-v2-scores.pkl", 'wb') as fIn:
#     pickle.dump(subscoreDict, fIn)
# os.symtem("gzip data/toy_data1k/cross-encoder-ms-marco-MiniLM-L-6-v2-scores.pkl")
# subsquer_Rep={qid:dataset.quer_Rep[int(qid)] for qid,query in Toyquerys}
# import pickle
# with open("data/toy_data1k/psuedo/log-doc-tf-topk-10.pkl", 'wb') as fIn:
#     pickle.dump(subsquer_Rep, fIn)
# import csv          
# def List2Tsv(inp: list,outputDir='output.tsv'):
#     with open(outputDir, 'w', newline='') as f_output:
#         tsv_output = csv.writer(f_output, delimiter='\t')
#         tsv_output.writerows(inp)


In [ ]:

# List2Tsv(Toyquerys,outputDir="toyQuery.tsv")


In [44]:
# queryText="what is the mechanism of action of fibrinolytic or thrombolytic drugs?"
ind=14
queryText=dataset[ind][0]
print(f"the query is :{queryText}")
top_k=10

queryExplicit,queryEMB=outputEmb(queryText,model=model,returnRep=True)
print(queryExplicit)
print(len(queryExplicit))

the query is :do roaches live outside
[('roach', 40.99), ('outside', 38.23), ('live', 25.82), ('inside', 25.61), ('cock', 23.81), ('##es', 23.35), ('out', 22.19), ('anywhere', 21.83), ('habitat', 21.51), ('found', 21.46), ('yes', 21.32), ('around', 21.25), ('predator', 21.12), ('survive', 20.99), ('living', 20.97), ('move', 20.95), ('bait', 20.83), ('do', 20.82), ('##e', 20.74), ('bug', 20.72), ('burr', 20.68), ('summer', 20.64), ('prison', 20.47), ('door', 20.4), ('tap', 20.35), ('cage', 20.31), ('kill', 20.29), ('snake', 20.29), ('trap', 20.27), ('cave', 20.27), ('presence', 20.25), ('safety', 20.22), ('pet', 20.22), ('garden', 20.21), ('fly', 20.21), ('no', 20.2), ('privacy', 20.19), ('window', 20.19), ('can', 20.18), ('bird', 20.18), ('location', 20.17), ('exterior', 20.16), ('housing', 20.14), ('dog', 20.13), ('stay', 20.13), ('shelter', 20.12), ('tunnel', 20.11), ('their', 20.1), ('animal', 20.09), ('south', 20.06), ('behavior', 20.05), ('apartment', 20.04), ('nest', 20.03), ('dw

In [45]:
# queryText="what is the mechanism of action of fibrinolytic or thrombolytic drugs?"
queryText=dataset[ind][0]
print(f"the query is :{queryText}")
top_k=10
queryExplicit,queryEMB=outputEmb(queryText,model=psuedo_model,returnRep=True)
print(queryExplicit)
print(len(queryExplicit))

the query is :do roaches live outside
[('roach', 5.07)]
1


In [ ]:
from collections import defaultdict
termWeight=defaultdict(float)
# idx=tokenizer.vocab[term]
count=0
for queryi in dataset.query_list:
    for termid in dataset.quer_Rep[queryi]:
        termWeight[reverse_voc[termid]]+=dataset.quer_Rep[queryi][termid]
termSorted=sorted(termWeight,key=lambda x:termWeight[x], reverse=True)
termWeight={key:termWeight[key]  for key in termSorted}

In [ ]:
for term,weight in queryExplicit:
    # print(term)
    # idx=tokenizer.vocab[term]
    if term in termWeight:
        print(term,termWeight[term])
    else:
        print(f"{term} not in ")

– not in 
" not in 
— not in 
’ not in 
” not in 
“ not in 
… not in 
‘ not in 
• not in 
[MASK] not in 
− not in 
##с not in 
→ not in 
″ not in 
##− not in 
##ل not in 
с not in 
′ not in 
ا not in 
##„ not in 
##о not in 
##р not in 
##а not in 
† not in 
ᆼ not in 
##е not in 
##止 not in 
##ا not in 
##ت not in 
##・ not in 
##т not in 
ಾ not in 
##и not in 
า not in 
##у not in 
ں not in 
##‡ not in 
##™ not in 
##亻 not in 
[unused77] not in 
ᅳ not in 
ী not in 
##¿ not in 
##ν not in 
ᅵ not in 
##， not in 
##ر not in 
##к not in 
##" not in 
##α not in 
##氵 not in 
##″ not in 
α not in 
##| not in 
##မ not in 
##‿ not in 
‑ not in 
##ε not in 
##( not in 
##₍ not in 
##⇄ not in 
م not in 
##॥ not in 
##অ not in 
[unused754] not in 
##十 not in 
##部 not in 
##′ not in 
‐ not in 
##» not in 
##м not in 
##م not in 
##་ not in 
##ه not in 
##巿 not in 
##[ not in 
[unused399] not in 
##弘 not in 
##艹 not in 
##τ not in 
##• not in 
ᅮ not in 
##辶 not in 
##& not in 
[unused533] not in 
##

KeyError: '’'

In [ ]:

query = dataset.query_list[ind]
print(query)
# dataset[ind]
# dataset.cortf_Rep[ind]
topic_Rep=dataset[ind][-2]
indTop=(-topic_Rep).argsort()[:100]
print(dataset[ind][0])

158771
do roaches live outside


In [ ]:
tripExp=[]
for term,weight in queryExplicit:
    # print(term)
    idx=tokenizer.vocab[term]
    # print(reverse_voc[t],topic_Rep[t],FinalTopic_Rep[t])
    tripExp.append([idx,weight,reverse_voc[idx],termWeight[reverse_voc[idx]],100*termWeight[reverse_voc[idx]]/dataset.cortf_Rep[idx],dataset.cortf_Rep[idx]])
    # if reverse_voc[idx] in termWeight:
    #     print(term,termWeight[reverse_voc[idx]])
tripExp=sorted(tripExp,key=lambda x:-x[1])
print(tripExp)

KeyError: '–'

In [ ]:

    # count+=dataset.quer_Rep[queryi][idx]

In [ ]:
tripExp=[]
for idx in indTop:
    # print(reverse_voc[t],topic_Rep[t],FinalTopic_Rep[t])
    tripExp.append([idx,reverse_voc[idx],topic_Rep[idx],dataset.cortf_Rep[idx],dataset.quer_Rep[query][idx]])
tripExp=sorted(tripExp,key=lambda x:-x[2])
print(tripExp)

[[1012, '.', 16.426168, 13294983.0, 16.426168413057834], [1996, 'the', 15.538049, 10955835.0, 15.538048558150376], [20997, 'roach', 11.074607, 900.6387, 11.074606531990986], [1010, ',', 8.995165, 10127588.0, 8.99516499031115], [2024, 'are', 8.147867, 2059837.2, 8.147867129923947], [2229, '##es', 7.8477626, 155180.61, 7.847762537473608], [2648, 'outside', 7.742402, 49930.445, 7.742402021815782], [1997, 'of', 7.742402, 7304573.5, 7.742402021815782], [1037, 'a', 7.6778636, 6784240.0, 7.677863500678211], [2003, 'is', 7.572503, 4729483.5, 7.572502985020385], [2188, 'home', 7.090077, 217457.62, 7.090076835776092], [1999, 'in', 7.049255, 5303185.0, 7.049254841255837], [101, '[CLS]', 6.931472, 6128684.5, 6.931471805599453], [102, '[SEP]', 6.931472, 6128684.5, 6.931471805599453], [3217, '##ro', 6.5792513, 44068.836, 6.579251212010101], [10338, 'cock', 6.5792513, 4021.3638, 6.579251212010101], [2030, 'or', 6.473891, 2385719.5, 6.473890696352275], [2027, 'they', 6.3561077, 613174.06, 6.3561076606

In [ ]:
idd=tokenizer.vocab['census']
print(reverse_voc[idd],topic_Rep[idd],dataset.cortf_Rep[idd],dataset.quer_Rep[query][idd])

census 0.0 38826.39 0


In [ ]:
indind=list(dataset.quer_Rep[query].keys())
values=list(dataset.quer_Rep[query].values())

In [ ]:
indsorted=sorted(dataset.quer_Rep[query],key=lambda x:-dataset.quer_Rep[query][x])

In [ ]:
dataset.quer_Rep[query][1012]

15.173405444562466